In [1]:
import pathlib

import pandas as pd
import pyarrow.parquet as pq

In [2]:
cell_type = "PBMC"

In [3]:
path_to_indexes = pathlib.Path(
    f"../indexes/{cell_type}/MultiClass_MLP_data_split_indexes.tsv"
).resolve()
file_path = pathlib.Path(
    f"../../../data/{cell_type}_preprocessed_sc_norm.parquet"
).resolve(strict=True)

In [4]:
indexes_df = pd.read_csv(path_to_indexes, sep="\t")
print(indexes_df.shape)
indexes_df.head()

(8318724, 2)


,labeled_data_index,label
0,6135062,train
1,5358238,train
2,3875296,train
3,7361184,train
4,4631111,train


In [5]:
# replace the index with the labeled_data index
indexes_df = indexes_df.set_index("labeled_data_index")
# sort the index
indexes_df = indexes_df.sort_index()
indexes_df.head()

,label
labeled_data_index,
0,val
1,test
2,test
3,test
4,test


In [6]:
df = pd.read_parquet(
    file_path,
    columns=["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"],
)
print(df.shape)
df.head()

(8318724, 1)


,oneb_Metadata_Treatment_Dose_Inhibitor_Dose
0,LPS_0.010_ug_per_ml_DMSO_0.025_%
1,LPS_0.010_ug_per_ml_DMSO_0.025_%
2,LPS_0.010_ug_per_ml_DMSO_0.025_%
3,LPS_0.010_ug_per_ml_DMSO_0.025_%
4,LPS_0.010_ug_per_ml_DMSO_0.025_%


In [7]:
# add data split label to the dataframe via index
df["data_split"] = indexes_df["label"]
df.head()

,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,data_split
0,LPS_0.010_ug_per_ml_DMSO_0.025_%,val
1,LPS_0.010_ug_per_ml_DMSO_0.025_%,test
2,LPS_0.010_ug_per_ml_DMSO_0.025_%,test
3,LPS_0.010_ug_per_ml_DMSO_0.025_%,test
4,LPS_0.010_ug_per_ml_DMSO_0.025_%,test


In [8]:
# get counts of each data split and treatment
grouped_df = pd.DataFrame(
    df.groupby(
        ["data_split", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    ).value_counts()
)
# melt the dataframe to get datasplits as columns
grouped_df = grouped_df.unstack(level=0)
# sort by treatment
grouped_df = grouped_df.sort_index()
grouped_df

0            \
data_split                                          holdout      test   
oneb_Metadata_Treatment_Dose_Inhibitor_Dose                             
DMSO_0.100_%_DMSO_0.025_%                           60328.0  214165.0   
DMSO_0.100_%_DMSO_1.000_%                           59877.0   88089.0   
DMSO_0.100_%_Z-VAD-FMK_100.000_uM                   56755.0   77058.0   
DMSO_0.100_%_Z-VAD-FMK_30.000_uM                    62967.0   84113.0   
Disulfiram_0.100_uM_DMSO_0.025_%                    67467.0   84953.0   
Disulfiram_1.000_uM_DMSO_0.025_%                    55502.0   87595.0   
Disulfiram_2.500_uM_DMSO_0.025_%                    58013.0   93462.0   
Flagellin_0.100_ug_per_ml_DMSO_0.025_%              53708.0       NaN   
Flagellin_1.000_ug_per_ml_DMSO_0.025_%              57805.0       NaN   
Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM       32379.0       NaN   
H2O2_100.000_nM_DMSO_0.025_%                        50374.0   89242.0   
H2O2_100.000_uM_DMSO_0.025_%                        64345.0   95072.0   
H2O2_100.000_uM_Disulfiram_1.000_uM                 55234.0   87045.0   
H2O2_100.000_uM_Z-VAD-FMK_100.000_uM                56132.0   78958.0   
LPS_0.010_ug_per_ml_DMSO_0.025_%                    53646.0  113320.0   
LPS_0.100_ug_per_ml_DMSO_0.025_%                    50909.0   78243.0   
LPS_1.000_ug_per_ml_DMSO_0.025_%                    53412.0       NaN   
LPS_10.000_ug_per_ml_DMSO_0.025_%                   58610.0   76679.0   
LPS_10.000_ug_per_ml_Disulfiram_0.100_uM            60095.0   77689.0   
LPS_10.000_ug_per_ml_Disulfiram_1.000_uM            46977.0   78208.0   
LPS_10.000_ug_per_ml_Disulfiram_2.500_uM            41651.0   69071.0   
LPS_10.000_ug_per_ml_Z-VAD-FMK_100.000_uM           56439.0   73457.0   
LPS_100.000_ug_per_ml_DMSO_0.025_%                  49634.0  113981.0   
LPS_Nigericin_1.000_ug_per_ml_1.000_uM_DMSO_0.0...  47673.0   75286.0   
LPS_Nigericin_1.000_ug_per_ml_10.000_uM_DMSO_0....  48545.0   76203.0   
LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Disulfi...  51889.0   62632.0   
LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-F...  53068.0   75534.0   
LPS_Nigericin_1.000_ug_per_ml_3.000_uM_DMSO_0.0...  51893.0       NaN   
LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_0...  53615.0   80366.0   
LPS_Nigericin_100.000_ug_per_ml_10.000_uM_DMSO_...  47281.0   68099.0   
LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_0...  52818.0   74389.0   
Media                                               57463.0  146096.0   
Thapsigargin_1.000_uM_DMSO_0.025_%                  62338.0   89771.0   
Thapsigargin_10.000_uM_DMSO_0.025_%                 51397.0   73019.0   
Topotecan_10.000_nM_DMSO_0.025_%                    51716.0   98465.0   
Topotecan_20.000_nM_DMSO_0.025_%                    60544.0   86302.0   
Topotecan_5.000_nM_DMSO_0.025_%                     65308.0   95147.0   

                                                              \
data_split                                             train   
oneb_Metadata_Treatment_Dose_Inhibitor_Dose                    
DMSO_0.100_%_DMSO_0.025_%                           171333.0   
DMSO_0.100_%_DMSO_1.000_%                            70471.0   
DMSO_0.100_%_Z-VAD-FMK_100.000_uM                    61646.0   
DMSO_0.100_%_Z-VAD-FMK_30.000_uM                     67290.0   
Disulfiram_0.100_uM_DMSO_0.025_%                     67962.0   
Disulfiram_1.000_uM_DMSO_0.025_%                     70076.0   
Disulfiram_2.500_uM_DMSO_0.025_%                     74770.0   
Flagellin_0.100_ug_per_ml_DMSO_0.025_%                   NaN   
Flagellin_1.000_ug_per_ml_DMSO_0.025_%                   NaN   
Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM            NaN   
H2O2_100.000_nM_DMSO_0.025_%                         71394.0   
H2O2_100.000_uM_DMSO_0.025_%                         76057.0   
H2O2_100.000_uM_Disulfiram_1.000_uM                  69637.0   
H2O2_100.000_uM_Z-VAD-FMK_100.000_uM                 63166.0   
LPS_0.010_ug_per_ml_DMSO_0.025_%                 